# PUPIL DATA

Raw data visualization with trials indicated by vertical lines.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = 'PupilData/PupilData_243356.csv'
df = pd.read_csv('PupilData/PupilData_243356.csv', header=0, names=['Timestamp', 'Scene', 'LeftPupil(mm)', 'RightPupil(mm)', 'LeftValid', 'RightValid'])

df.head()
df['LeftPupil(mm)'] = pd.to_numeric(df['LeftPupil(mm)'], errors='coerce')
df['RightPupil(mm)'] = pd.to_numeric(df['RightPupil(mm)'], errors='coerce')

plt.figure(figsize=(12, 6))
plt.plot(df['LeftPupil(mm)'], label='Left Pupil', color='skyblue')
plt.plot(df['RightPupil(mm)'], label='Right Pupil', color='lightgreen')

for i in range(1, len(df)):
    if (df['Scene'].iloc[i-1] == 'AvatarSceneHappy' or df['Scene'].iloc[i-1] == 'AvatarSceneNeutral') and df['Scene'].iloc[i] == 'BaselineScene':
        plt.axvline(x=i, color='r', linestyle='--', alpha=0.5)

plt.title('Raw Pupil Traces of 243356')
plt.xlabel('Sample Index')
plt.ylabel('Pupil Size (mm)')
plt.legend()
plt.show()

### PUPIL DATA PREPROCESSING

Checking pupil data validity and correlations.

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

data_folder = 'PupilData/'
summary = []

for file_path in glob(os.path.join(data_folder, 'PupilData_*.csv')):
    try:
        df = pd.read_csv(file_path)

        df['LeftPupil(mm)'] = pd.to_numeric(df['LeftPupil(mm)'], errors='coerce')
        df['RightPupil(mm)'] = pd.to_numeric(df['RightPupil(mm)'], errors='coerce')
        df['LeftValid'] = df['LeftValid'].astype(str).str.lower().map({'true': True, 'false': False})
        df['RightValid'] = df['RightValid'].astype(str).str.lower().map({'true': True, 'false': False})

        participant_id = os.path.basename(file_path).split('_')[1].replace('.csv', '')

        left_valid_rate = df['LeftValid'].mean() * 100
        right_valid_rate = df['RightValid'].mean() * 100

        valid_data = df[df['LeftValid'] & df['RightValid']].copy()
        valid_data = valid_data.replace(0, np.nan).dropna(subset=['LeftPupil(mm)', 'RightPupil(mm)'])

        correlation = valid_data['LeftPupil(mm)'].corr(valid_data['RightPupil(mm)'])

        summary.append({
            'Participant': participant_id,
            'Left Validity (%)': round(left_valid_rate, 2),
            'Right Validity (%)': round(right_valid_rate, 2),
            'Left-Right Correlation': round(correlation, 2) if not np.isnan(correlation) else None
        })
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

summary_df = pd.DataFrame(summary)
summary_df.set_index('Participant', inplace=True)

participants = summary_df.index.astype(str)

fig, axs = plt.subplots(3, 1, figsize=(14, 14), gridspec_kw={'hspace': 0.4})

# 1. Left vs Right Validity (Clustered Bar Chart)
x = range(len(participants))
bar_width = 0.35

axs[0].bar(x, summary_df['Left Validity (%)'], width=bar_width, label='Left Validity', color='skyblue')
axs[0].bar([i + bar_width for i in x], summary_df['Right Validity (%)'], width=bar_width, label='Right Validity', color='lightgreen')
axs[0].axhline(95, color='red', linestyle='--', label='95% Threshold')
axs[0].set_title('Validity of Pupil Data by Participant')
axs[0].set_ylabel('Validity (%)')
axs[0].set_xticks([i + bar_width / 2 for i in x])
axs[0].set_xticklabels(participants, rotation=90)
axs[0].legend()

# 2. Left-Right Pupil Correlation (Bar Chart)
axs[1].bar(participants, summary_df['Left-Right Correlation'], color='coral')
axs[1].axhline(0.5, color='red', linestyle='--', label='Threshold (r=0.5)')
axs[1].set_title('Correlation Between Left and Right Pupil Size')
axs[1].set_ylabel('Pearson Correlation (r)')
axs[1].legend()

# 3. Scatter: Validity vs Correlation (Bubble Plot)
sns.scatterplot(
    data=summary_df.reset_index(),
    x='Left Validity (%)',
    y='Left-Right Correlation',
    size='Right Validity (%)',
    sizes=(50, 300),
    hue='Right Validity (%)',
    palette='viridis',
    ax=axs[2]
)
axs[2].axhline(0.5, color='red', linestyle='--')
axs[2].axvline(95, color='blue', linestyle='--')
axs[2].set_title('Left Validity vs Correlation (Bubble = Right Validity)')
axs[2].set_xlabel('Left Validity (%)')
axs[2].set_ylabel('Left-Right Correlation')
axs[2].legend(title='Right Validity (%)')

plt.tight_layout()
plt.show()


Preprocessing remaining 22 files (4 files excluded with the previous step)

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, savgol_filter
import matplotlib.pyplot as plt
import os
import pandas as pd

def preprocess_left_eye_pupil(file_path, surrounding_ms=200, fs=10, plot=True):
    # === Load and basic filtering ===
    df = pd.read_csv(file_path)
    df['LeftValid'] = df['LeftValid'].astype(str).str.lower() == 'true'
    df['LeftPupil(mm)'] = pd.to_numeric(df['LeftPupil(mm)'], errors='coerce')
    df.loc[~df['LeftValid'], 'LeftPupil(mm)'] = np.nan
    df_raw = df.copy()

    # === Implausible values removed ===
    df.loc[(df['LeftPupil(mm)'] < 1.5) | (df['LeftPupil(mm)'] > 7.0), 'LeftPupil(mm)'] = np.nan

    # === Blink-surrounding trimming ===
    surrounding = int((surrounding_ms / 1000) * fs)
    blink_indices = df[df['LeftPupil(mm)'].isna()].index
    for i in blink_indices:
        df.loc[max(i - surrounding, 0):min(i + surrounding, len(df) - 1), 'LeftPupil(mm)'] = np.nan

    # === Remove spikes & high acceleration ===
    diff = df['LeftPupil(mm)'].diff().abs()
    df.loc[diff > 0.5, 'LeftPupil(mm)'] = np.nan

    accel = None
    if not df.empty:
        accel = df['LeftPupil(mm)'].diff().diff().abs()
    
    df.loc[accel > 0.3, 'LeftPupil(mm)'] = np.nan

    # === Interpolation ===
    df['LeftPupil(mm)'] = df['LeftPupil(mm)'].interpolate(method='spline', order=2, limit_direction='both')

    # === Savitzky-Golay smoothing ===
    df['LeftPupil(mm)'] = savgol_filter(df['LeftPupil(mm)'].fillna(method='pad'), window_length=5, polyorder=2)

    # === Butterworth lowpass filtering (balanced) ===
    def butter_lowpass_filter(data, cutoff=0.6, fs=10, order=3):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return filtfilt(b, a, data)

    df['LeftPupilFiltered(mm)'] = butter_lowpass_filter(df['LeftPupil(mm)'], cutoff=0.6, fs=fs)

    if plot:
        plt.figure(figsize=(16, 6))
        plt.plot(df_raw['LeftPupil(mm)'], label='Left Eye (Raw)', color='lightgreen')
        plt.plot(df['LeftPupilFiltered(mm)'], label='Left Eye (Filtered)', color='coral')
        plt.title('Left Eye Pupil Size – Filtered')
        plt.xlabel('Sample Index')
        plt.ylabel('Pupil Size (mm)')
        plt.legend()
        plt.tight_layout()
        plt.show()

    return df

folder_path = 'PupilData'

all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and '_corrupted' not in f and '_outlier' not in f]

cleaned_data = {}
for file in all_files:
    participant_id = file.replace('.csv', '').split('_')[-1]
    file_path = os.path.join(folder_path, file)

    try:
        cleaned_df = preprocess_left_eye_pupil(file_path, plot=True, fs=10)
        cleaned_data[participant_id] = cleaned_df
    except Exception as e:
        print(f"⚠️ Skipping {file} due to error: {e}")

### BVP DATA

Visualization of raw signal

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(
    "../Trial1/243349.txt",
    skiprows=9,
    header=None,
    usecols=[0, 1, 2],
    names=['min', 'CH1', 'CH40']
)

df['min'] = df['min'].astype(float)
df['time_sec'] = df['min'] * 60

plt.figure(figsize=(15, 4))
plt.plot(df['time_sec'], df['CH1'], lw=0.5, color='red')
plt.xlabel('Time (s)')
plt.ylabel('PPG (Volts)')
plt.show()

Preprocessing with neurokit2

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
import re


parent_dir = "./PulseData"
trial_types = ["BaselineAnxietyHappy", "BaselineAnxietyNeutral", "BaselineNeutralHappy", "BaselineNeutralNeutral"]

def process_ppg_file(filepath, trial_type, participant_id, show_plot=False):
    df = pd.read_csv(
        filepath,
        skiprows=9,
        header=None,
        usecols=[0, 1, 2],
        names=['min', 'CH1', 'CH40']
    )
    df['min'] = df['min'].astype(float)
    df['time_sec'] = df['min'] * 60

    fs = 2000
    raw_signal = df['CH1'].values
    time = df['time_sec'].values

    TRIM_SEC = 3
    start_time = time[0] + TRIM_SEC
    end_time = time[-1] - TRIM_SEC
    mask = (time >= start_time) & (time <= end_time)
    time_trimmed = time[mask]
    signal_trimmed = raw_signal[mask]

    try:
        plt.rcParams['figure.figsize'] = (12, 8)  # Wider and taller
        signals, info = nk.ppg_process(signal_trimmed, sampling_rate=fs, report="test.html")
    except Exception as e:
        print(f"❌ Error processing full signal: {e}")
        return None

    if show_plot:
        plt.figure(figsize=(15, 4))
        plt.plot(time_trimmed, signal_trimmed, alpha=0.4, label="Raw Trimmed")
        plt.plot(time_trimmed, signals['PPG_Clean'], label="PPG Clean", linewidth=1)
        plt.xlabel("Time (s)")
        plt.ylabel("PPG")
        plt.legend()
        plt.title("Full PPG Signal (Cleaned)")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    baseline_start = start_time
    baseline_end = start_time + 60
    task_start = baseline_end
    task_end = end_time - 30
    avatar_start = task_end
    avatar_end = end_time

    def get_mask(scene_start, scene_end, trim_start=True, trim_end=True, trim_sec=3):
        start = scene_start + (trim_sec if trim_start else 0)
        end = scene_end - (trim_sec if trim_end else 0)
        return (time_trimmed >= start) & (time_trimmed < end)

    mask_baseline = get_mask(baseline_start, baseline_end, trim_start=False, trim_end=True)
    mask_task = get_mask(task_start, task_end, trim_start=True, trim_end=True)
    mask_avatar = get_mask(avatar_start, avatar_end, trim_start=True, trim_end=False)

    # === 5. Calculate HR & RMSSD from PPG_Rate ===
    def compute_hrv_metrics(rate_series, sampling_rate=fs):
        if rate_series.empty or len(rate_series) < sampling_rate * 5 / 1000:
            return {"HR_mean": np.nan, "RMSSD": np.nan, "SDNN": np.nan}
        rate_series = rate_series.dropna()
        if rate_series.empty:
            return {"HR_mean": np.nan, "RMSSD": np.nan, "SDNN": np.nan}
        
        hrv_df = nk.hrv(info, sampling_rate=sampling_rate, show=False)
        return {
            "HR_mean": rate_series.mean(),
            "RMSSD": hrv_df["HRV_RMSSD"].values[0] if "HRV_RMSSD" in hrv_df else np.nan,
            "SDNN": hrv_df["HRV_SDNN"].values[0] if "HRV_SDNN" in hrv_df else np.nan
        }

    try:
        hrv_baseline = compute_hrv_metrics(signals['PPG_Rate'][mask_baseline])
        hrv_task = compute_hrv_metrics(signals['PPG_Rate'][mask_task])
        hrv_avatar = compute_hrv_metrics(signals['PPG_Rate'][mask_avatar])
    except Exception as e:
        print(f"❌ Error computing HRV per segment: {e}")
        return None

    trial_parts = re.split(r'(?=[A-Z])', trial_type)
    task_type = trial_parts[2] + "Scene"
    avatar_type = "AvatarScene" + trial_parts[3]

    summary = {
        "participant": participant_id,
        "trial_type": trial_type,
        "task_type": task_type,
        "avatar_type": avatar_type,
        "baseline_HR": hrv_baseline["HR_mean"],
        "baseline_SDNN": hrv_baseline["SDNN"],
        "baseline_RMSSD": hrv_baseline["RMSSD"],
        "task_HR": hrv_task["HR_mean"],
        "task_SDNN": hrv_task["SDNN"],
        "task_RMSSD": hrv_task["RMSSD"],
        "avatar_HR": hrv_avatar["HR_mean"],
        "avatar_SDNN": hrv_avatar["SDNN"],
        "avatar_RMSSD": hrv_avatar["RMSSD"],
        "delta_HR_task_minus_baseline": hrv_task["HR_mean"] - hrv_baseline["HR_mean"] if pd.notna(hrv_task["HR_mean"]) and pd.notna(hrv_baseline["HR_mean"]) else np.nan,
        "delta_HR_avatar_minus_task": hrv_avatar["HR_mean"] - hrv_task["HR_mean"] if pd.notna(hrv_avatar["HR_mean"]) and pd.notna(hrv_task["HR_mean"]) else np.nan,
        "delta_RMSSD_task_minus_baseline": hrv_task["RMSSD"] - hrv_baseline["RMSSD"] if pd.notna(hrv_task["RMSSD"]) and pd.notna(hrv_baseline["RMSSD"]) else np.nan,
        "delta_RMSSD_avatar_minus_task": hrv_avatar["RMSSD"] - hrv_task["RMSSD"] if pd.notna(hrv_avatar["RMSSD"]) and pd.notna(hrv_task["RMSSD"]) else np.nan
    }

    return summary

# --- Batch processing ---
summary_list = []
for trial_type in trial_types:
    folder = os.path.join(parent_dir, trial_type)
    if not os.path.exists(folder):
        print(folder + " does not exist")
        continue
    for fname in os.listdir(folder):
        if not fname.endswith('.txt'):
            continue
        participant_id = fname.replace('.txt', '')[:6]
        
        filepath = os.path.join(folder, fname)
        print(f"Processing {filepath} ...")
        summary = process_ppg_file(filepath, trial_type, participant_id, show_plot=True)
        summary_list.append(summary)

# --- Save summary DataFrame ---
summary_df = pd.DataFrame(summary_list)
summary_df.to_csv("ppg_summary_results.csv", index=False)

### STAI-S SCORES CALCULATION

In [ ]:
import pandas as pd
import re

input_csv = 'Experiment1 - Form Responses.csv'
output_csv = 'STAI_Scores.csv'

score_map = {
    '全くない / Not at all': 1,
    '少しある / Somewhat': 2,
    'かなりある / Moderately so': 3,
    '非常にある / Very much so': 4
}

reverse_scored_items = [1, 2, 5, 8, 10, 11, 15, 16, 19, 20]

try:
    df = pd.read_csv(input_csv)
    output_data = {'ParticipantID': df['学生ID / StudentID']}

    for i in range(1, 6):
        stai_columns = {}
        for col in df.columns:
            match = re.match(rf'STAI-S #{i} \[(.+?)\s*/\s*(.+?)\]', col)
            if match:
                english_text = match.group(2).strip()
                item_mapping_text_to_number = {
                    "I feel calm.": 1,
                    "I feel secure.": 2,
                    "I am tense.": 3,
                    "I feel strained.": 4,
                    "I feel at ease.": 5,
                    "I feel upset.": 6,
                    "I am worrying over possible misfortunes.": 7,
                    "I feel satisfied.": 8,
                    "I feel frightened.": 9,
                    "I feel comfortable.": 10,
                    "I feel self-confident.": 11,
                    "I feel nervous.": 12,
                    "I am jittery.": 13,
                    "I feel indecisive.": 14,
                    "I am relaxed.": 15,
                    "I feel content.": 16,
                    "I am worried.": 17,
                    "I feel confused.": 18,
                    "I feel steady.": 19,
                    "I feel pleasant.": 20
                }
                item_number = item_mapping_text_to_number.get(english_text)
                if item_number is not None:
                     stai_columns[col] = item_number

        stai_df = df[list(stai_columns.keys())].copy()

        stai_df.replace(score_map, inplace=True)

        for col, item_num in stai_columns.items():
            if item_num in reverse_scored_items:
                stai_df[col] = 5 - stai_df[col]

        output_data[f'STAI-S Score {i}'] = stai_df.sum(axis=1)

    stai_df = pd.DataFrame(output_data)
except FileNotFoundError:
    print(f"Error: Input file not found at {input_csv}")
except KeyError as e:
    print(f"Error: Missing expected column in {input_csv}: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Separate participants into anxiety grous based on baseline STAI-S

In [ ]:
try:
    df_scores = stai_df
    median_stai_score_1 = df_scores['STAI-S Score 1'].median()

    print(f"Median STAI-S Score 1: {median_stai_score_1}")

    df_scores['Anxiety_Group'] = df_scores['STAI-S Score 1'].apply(
        lambda score: 'Higher Anxiety' if score > median_stai_score_1 else 'Lower Anxiety'
    )

    print("\nDataFrame with Anxiety Groups:")
    print(df_scores.head())
    df_scores.to_csv('STAI_Scores_with_Groups.csv', index=False)
except FileNotFoundError:
    print(f"Error: Input file not found at {input_csv}")
except KeyError as e:
    print(f"Error: Missing expected column in {input_csv}: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## MAIN ANALYSIS

Anxiety validation by pupil data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

def classify_phase(scene_name):
    if "BaselineScene" in scene_name:
        return "Baseline", None
    elif "AnxietyScene" in scene_name:
        return "Scene", "Anxiety"
    elif "NeutralScene" in scene_name:
        return "Scene", "NeutralScene"
    elif "AvatarSceneHappy" in scene_name:
        return "Avatar", "Happy"
    elif "AvatarSceneNeutral" in scene_name:
        return "Avatar", "Neutral"
    return "Other", None

scene_counters = defaultdict(int)
trial_metrics = []

trial_metrics = []

for participant_id, df in cleaned_data.items():
    current_trial = None
    current_phase = None
    trial_num = 1
    
    baseline_data = []
    scene_data = []
    avatar_data = []
    
    for _, row in df.iterrows():
        scene = row['Scene']
        pupil_size = row['LeftPupilFiltered(mm)']
        
        scene_type, condition = classify_phase(scene)
        
        # Case 1: Baseline scene starts a new trial
        if scene_type == "Baseline":
            if current_phase == "Avatar":
                # Save completed trial
                if baseline_data and scene_data and avatar_data:
                    trial_metrics.append({
                        "Participant": participant_id,
                        "Trial": trial_num,
                        "SceneType": "Anxiety" if "Anxiety" in scene_data[0]['scene'] else "Neutral",
                        "AvatarType": "Happy" if "Happy" in avatar_data[0]['scene'] else "Neutral",
                        "BaselineMean": np.mean([d['pupil'] for d in baseline_data]),
                        "SceneMean": np.mean([d['pupil'] for d in scene_data]),
                        "AvatarMean": np.mean([d['pupil'] for d in avatar_data]),
                        "SceneΔ": np.mean([d['pupil'] for d in scene_data]) - np.mean([d['pupil'] for d in baseline_data]),
                        "AvatarΔ": np.mean([d['pupil'] for d in avatar_data]) - np.mean([d['pupil'] for d in baseline_data]),
                        "Scene→AvatarΔ": np.mean([d['pupil'] for d in avatar_data]) - np.mean([d['pupil'] for d in scene_data]),
                        "BaselineDuration": len(baseline_data),
                        "SceneDuration": len(scene_data),
                        "AvatarDuration": len(avatar_data)
                    })
                    trial_num += 1
                
                # Reset for new trial
                baseline_data = []
                scene_data = []
                avatar_data = []
            
            current_phase = "Baseline"
            baseline_data.append({"scene": scene, "pupil": pupil_size})
        
        # Case 2: Anxiety/Neutral scene (only record if we're in a trial)
        elif scene_type == "Scene" and current_phase in ["Baseline", "Scene"]:
            current_phase = "Scene"
            scene_data.append({"scene": scene, "pupil": pupil_size})
        
        # Case 3: Avatar scene (only record if we were just in Scene phase)
        elif scene_type == "Avatar" and current_phase == "Scene":
            current_phase = "Avatar"
            avatar_data.append({"scene": scene, "pupil": pupil_size})
        
    
    if baseline_data and scene_data and avatar_data:
        trial_metrics.append({
            "Participant": participant_id,
            "Trial": trial_num,
            "SceneType": "Anxiety" if "Anxiety" in scene_data[0]['scene'] else "Neutral",
            "AvatarType": "Happy" if "Happy" in avatar_data[0]['scene'] else "Neutral",
            "BaselineMean": np.mean([d['pupil'] for d in baseline_data]),
            "SceneMean": np.mean([d['pupil'] for d in scene_data]),
            "AvatarMean": np.mean([d['pupil'] for d in avatar_data]),
            "SceneΔ": np.mean([d['pupil'] for d in scene_data]) - np.mean([d['pupil'] for d in baseline_data]),
            "AvatarΔ": np.mean([d['pupil'] for d in avatar_data]) - np.mean([d['pupil'] for d in baseline_data]),
            "Scene→AvatarΔ": np.mean([d['pupil'] for d in avatar_data]) - np.mean([d['pupil'] for d in scene_data]),
            "BaselineDuration": len(baseline_data),
            "SceneDuration": len(scene_data),
            "AvatarDuration": len(avatar_data)
        })

trial_df = pd.DataFrame(trial_metrics)
df = trial_df
df['Participant'] = df['Participant'].astype('category')
df['SceneType'] = df['SceneType'].astype('category')
df['AvatarType'] = df['AvatarType'].astype('category')

plt.figure(figsize=(8, 6))
sns.boxplot(x='SceneType', y='SceneΔ', data=df, palette='Set2')
sns.stripplot(x='SceneType', y='SceneΔ', data=df, color='black', alpha=0.5, jitter=True)
plt.title('Pupil Dilation (Scene – Baseline) by Scene Type')
plt.ylabel('SceneΔ (mm)')
plt.xlabel('Scene Type')
plt.tight_layout()
plt.show()

In [ ]:
trial_df['AppearanceOrder'] = trial_df.groupby(['Participant', 'SceneType'])['Trial'].rank()
trial_df['AppearanceOrder'] = trial_df['AppearanceOrder'].map({1: 'First', 2: 'Second'})

plt.figure(figsize=(12, 6))
sns.boxplot(
    x='SceneType',
    y='SceneΔ',
    hue='AppearanceOrder',
    data=trial_df,
    palette={'First': '#1f77b4', 'Second': '#ff7f0e'},
    whis=1.5
)
plt.title('Pupil Dilation by Scene Type and Exposure Order')
plt.ylabel('Δ Pupil Size (mm)')
plt.xlabel('Scene Type')
plt.legend(title='Exposure Order')
sns.despine()
plt.show()

Statistical analysis of Task Scene Type in Pupil Data

In [ ]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
import statsmodels.formula.api as smf

first_exposure = trial_df[trial_df['AppearanceOrder']=='First'].groupby('Participant')['SceneΔ'].mean()
second_exposure = trial_df[trial_df['AppearanceOrder']=='Second'].groupby('Participant')['SceneΔ'].mean()
print(wilcoxon(first_exposure, second_exposure))

anxiety = trial_df[trial_df['SceneType'] == 'Anxiety']['SceneΔ']
neutral = trial_df[trial_df['SceneType'] == 'Neutral']['SceneΔ']
wilcoxon(anxiety, neutral)

def rank_biserial(x, y):
    u_stat = mannwhitneyu(x, y).statistic
    n1, n2 = len(x), len(y)
    return 1 - (2 * u_stat / (n1 * n2))  # r = 1 - (2U/nm)

r = rank_biserial(anxiety, neutral)
print(f"Rank-Biserial r = {r:.3f}")

def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(((nx-1)*np.std(x, ddof=1)**2 + (ny-1)*np.std(y, ddof=1)**2) / (nx + ny - 2))
    return (np.mean(x) - np.mean(y)) / pooled_std

print("Cohen’s d (Anxiety vs. Neutral):", cohens_d(anxiety, neutral))

model = smf.mixedlm("SceneΔ ~ SceneType", data=trial_df, 
                    groups=trial_df["Participant"]).fit()
print(model.summary())

Anxiety validation by pulse data

In [ ]:
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = summary_df

anxiety_group = df[df['task_type'] == 'AnxietyScene']
neutral_group = df[df['task_type'] == 'NeutralScene']

print("Mean HR increase (Anxiety):", anxiety_group['delta_HR_task_minus_baseline'].mean())
print("Mean HR increase (Neutral):", neutral_group['delta_HR_task_minus_baseline'].mean())

print("Mean RMSSD change (Anxiety):", anxiety_group['delta_RMSSD_task_minus_baseline'].mean())
print("Mean RMSSD change (Neutral):", neutral_group['delta_RMSSD_task_minus_baseline'].mean())

print("Mean SDNN change (Anxiety):", (anxiety_group['task_SDNN'] - anxiety_group['baseline_SDNN']).mean())
print("Mean SDNN change (Neutral):", (neutral_group['task_SDNN'] - neutral_group['baseline_SDNN']).mean())

t_stat, p_val = ttest_rel(
    anxiety_group['delta_HR_task_minus_baseline'].dropna(),
    neutral_group['delta_HR_task_minus_baseline'].dropna()
)
print(f"Paired t-test: t={t_stat:.2f}, p={p_val:.3f}")

plt.figure(figsize=(10, 6))

boxprops = dict(linestyle='-', linewidth=2, color='skyblue')
medianprops = dict(linestyle='-', linewidth=0.5, color='red')
whiskerprops = dict(linestyle='--', linewidth=1.5, color='#2ca02c')

bp = plt.boxplot([
    anxiety_group['delta_HR_task_minus_baseline'].dropna(),
    neutral_group['delta_HR_task_minus_baseline'].dropna()
], 
labels=['Anxiety', 'Neutral'],
patch_artist=True,
boxprops=dict(facecolor='skyblue', **boxprops),
medianprops=medianprops,
capprops=dict(linewidth=2),
widths=0.6)

for i, group in enumerate([anxiety_group, neutral_group]):
    y = group['delta_HR_task_minus_baseline'].dropna()
    x = np.random.normal(i+1, 0.04, size=len(y))
    plt.plot(x, y, 'o', color='#7f7f7f', alpha=0.6, markersize=8, markeredgewidth=0.5, markeredgecolor='white')

plt.ylabel('HR Change (bpm)\nTask Scene - Baseline', fontsize=12, labelpad=10)
plt.xlabel('Scene Type', fontsize=12, labelpad=10)
sns.despine()

plt.tight_layout()
plt.show()

### AVATAR EFFECTS

In [ ]:
df = trial_df

# === Boxplot: Scene→AvatarΔ by AvatarType ===
plt.figure(figsize=(8, 6))
sns.boxplot(x='AvatarType', y='Scene→AvatarΔ', data=df, palette='Set2')
sns.stripplot(x='AvatarType', y='Scene→AvatarΔ', data=df, color='black', alpha=0.5, jitter=True)
plt.title('Pupil Change (Avatar – Scene) by Avatar Type')
plt.ylabel('Scene→AvatarΔ (mm)')
plt.xlabel('Avatar Type')
plt.tight_layout()
plt.show()

# === Lineplot: Each participant's 4 trials ===
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='Trial', y='Scene→AvatarΔ', hue='Participant', marker='o', legend=False)
plt.title('Within-Subject Change from Scene to Avatar')
plt.ylabel('Scene→AvatarΔ (mm)')
plt.xlabel('Trial')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
sns.barplot(x='SceneType', y='Scene→AvatarΔ', hue='AvatarType', data=df, ci='sd')
plt.title('Scene × Avatar Interaction on Pupil Change (Avatar – Scene)')
plt.ylabel('Δ Pupil Size (mm)')
plt.xlabel('Scene Type')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import ttest_rel, wilcoxon

happy = df[df['AvatarType'] == 'Happy']['Scene→AvatarΔ']
neutral = df[df['AvatarType'] == 'Neutral']['Scene→AvatarΔ']
print(len(happy))
print(len(neutral))

t_stat, p_val = ttest_rel(happy, neutral)
print(f"Paired t-test: t={t_stat:.2f}, p={p_val:.3f}")

w_stat, w_p = wilcoxon(happy, neutral)
print(f"Wilcoxon: stat={w_stat}, p={w_p:.3f}")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df = summary_df

plt.figure(figsize=(8,6))
sns.boxplot(
    x='task_type',
    y='delta_HR_avatar_minus_task',
    hue='avatar_type',
    data=df,
    palette='Set3'
)
plt.xlabel('Task Type')
plt.ylabel('HR Reduction (Avatar - Task)')
plt.legend(title='Avatar Type')
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import ttest_rel

happy = df[df['avatar_type'] == 'AvatarSceneHappy']
neutral = df[df['avatar_type'] == 'AvatarSceneNeutral']

print("Mean HR reduction (Happy avatar):", happy['delta_HR_avatar_minus_task'].mean())
print("Mean HR reduction (Neutral avatar):", neutral['delta_HR_avatar_minus_task'].mean())

print("Mean RMSSD change (Anxiety):", happy['delta_RMSSD_avatar_minus_task'].mean())
print("Mean RMSSD change (Neutral):", neutral['delta_RMSSD_avatar_minus_task'].mean())

print("Mean SDNN change (Anxiety):", (happy['avatar_SDNN'] - happy['task_SDNN']).mean())
print("Mean SDNN change (Neutral):", (neutral['avatar_SDNN'] - neutral['task_SDNN']).mean())

t, p = ttest_rel(
    happy['delta_HR_avatar_minus_task'].values,
    neutral['delta_HR_avatar_minus_task'].values
)
print(f"Paired t-test: t={t:.2f}, p={p:.3f}")

Analysis by anxiety groups based on initial STAI-S

In [ ]:
stai_df = pd.read_csv('STAI_Scores_with_Groups.csv') 
stai_df['ParticipantID'] = stai_df['ParticipantID'].astype(str)

merged_dfs = []

for pid, df in cleaned_data.items():
    df = df.copy()
    df['ParticipantID'] = pid
    df = df.merge(stai_df[['ParticipantID', 'Anxiety_Group']], on='ParticipantID', how='left')
    merged_dfs.append(df)

full_df = pd.concat(merged_dfs, ignore_index=True)

In [ ]:
import pandas as pd

BASELINE_SCENE = 'BaselineScene'
TASK_SCENES = ['AnxietyScene', 'NeutralScene']
AVATAR_SCENES = ['AvatarSceneHappy', 'AvatarSceneNeutral']

def assign_trial_numbers(df):
    df = df.copy()
    df['Trial'] = -1
    trial = -1
    prev_scene = None
    for idx, row in df.iterrows():
        if row['Scene'] == 'BaselineScene' and prev_scene != 'BaselineScene':
            trial += 1
        df.at[idx, 'Trial'] = trial
        prev_scene = row['Scene']
    return df

full_df = full_df.groupby('ParticipantID', group_keys=False).apply(assign_trial_numbers)

trial_summaries = []

for (pid, trial), group in full_df.groupby(['ParticipantID', 'Trial']):
    anxiety_group = group['Anxiety_Group'].iloc[0]

    stai_row = stai_df[stai_df['ParticipantID'] == str(pid)]
    if stai_row.empty or trial < 0 or trial >= 4:
        continue

    pre_stai = stai_row[f'STAI-S Score {trial+1}'].values[0]
    post_stai = stai_row[f'STAI-S Score {trial+2}'].values[0]
    stai_change = post_stai - pre_stai
    
    scenes = group['Scene'].unique()
    # Extract baseline, task, and avatar scenes
    baseline = group[group['Scene'] == BASELINE_SCENE]['LeftPupilFiltered(mm)']
    task = group[group['Scene'].isin(TASK_SCENES)]['LeftPupilFiltered(mm)']
    avatar = group[group['Scene'].isin(AVATAR_SCENES)]['LeftPupilFiltered(mm)']
    
    task_type = group[group['Scene'].isin(TASK_SCENES)]['Scene'].iloc[0] if not task.empty else None
    avatar_type = group[group['Scene'].isin(AVATAR_SCENES)]['Scene'].iloc[0] if not avatar.empty else None
    
    baseline_mean = baseline.mean() if not baseline.empty else None
    task_mean = task.mean() if not task.empty else None
    avatar_mean = avatar.mean() if not avatar.empty else None

    task_change = task_mean - baseline_mean if task_mean is not None and baseline_mean is not None else None
    avatar_change = avatar_mean - baseline_mean if avatar_mean is not None and baseline_mean is not None else None

    trial_summaries.append({
        'ParticipantID': pid,
        'Anxiety_Group': anxiety_group,
        'Trial': trial,
        'TaskType': task_type,
        'AvatarType': avatar_type,
        'BaselineMean': baseline_mean,
        'TaskMean': task_mean,
        'AvatarMean': avatar_mean,
        'TaskChange': task_change,
        'AvatarChange': avatar_change,
        'STAI_Pre': pre_stai,
        'STAI_Post': post_stai,
        'STAI_Change': stai_change
    })

summary_df = pd.DataFrame(trial_summaries)

In [109]:
summary_df['AvatarShift'] = summary_df['AvatarMean'] - summary_df['TaskMean']
summary_df.to_csv("summary_group_pupil.csv", index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.boxplot(
    x='AvatarType',
    y='AvatarChange',
    hue='Anxiety_Group',
    data=summary_df
)
plt.title("Pupil Change (AvatarChange) by Avatar Type and Anxiety Group")
plt.show()


In [ ]:
import statsmodels.formula.api as smf

# For AvatarShift
model_avatarshift = smf.mixedlm(
    "AvatarShift ~ AvatarType * TaskType * Anxiety_Group",
    data=summary_df,
    groups=summary_df["ParticipantID"]
)
result_avatarshift = model_avatarshift.fit(method='nm')
print(result_avatarshift.summary())

# For STAI_Change
model_staichange = smf.mixedlm(
    "STAI_Change ~ AvatarType * TaskType * Anxiety_Group",
    data=summary_df,
    groups=summary_df["ParticipantID"]
)
result_staichange = model_staichange.fit(method='lbfgs')
print(result_staichange.summary())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.boxplot(x='Anxiety_Group', y='AvatarShift', data=summary_df)
sns.stripplot(x='Anxiety_Group', y='AvatarShift', data=summary_df, color='k', alpha=0.4, jitter=0.2)
plt.title('Avatar Shift by Anxiety Group')
plt.ylabel('Avatar Shift (AvatarMean - TaskMean)')
plt.xlabel('Anxiety Group')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.pointplot(
    x='AvatarType',
    y='STAI_Change',
    hue='TaskType',
    data=summary_df,
    dodge=True,
    capsize=.1,
    errwidth=1,
    palette='Set2'
)
plt.title('STAI Change by Avatar Type and Task Type')
plt.ylabel('STAI Change')
plt.xlabel('Avatar Type')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
sns.pointplot(
    x='TaskType',
    y='STAI_Change',
    hue='Anxiety_Group',
    data=summary_df,
    dodge=True,
    capsize=.1,
    errwidth=1,
    palette='Set1'
)
plt.title('STAI Change by Task Type and Anxiety Group')
plt.ylabel('STAI Change')
plt.xlabel('Task Type')
plt.tight_layout()
plt.show()

In [120]:
import pandas as pd

file1 = pd.read_csv("summary_group_pupil.csv") 
file2 = pd.read_csv("ppg_summary_results.csv")

merged = pd.merge(
    file1,
    file2,
    how='left',
    left_on=['ParticipantID', 'TaskType', 'AvatarType'],
    right_on=['participant', 'task_type', 'avatar_type']
)

In [ ]:
from dython.nominal import associations

subset = df[['Anxiety_Group', 'TaskType', 'AvatarType', 'STAI_Pre', 'STAI_Post', 'STAI_Change',
    'TaskChange', 'AvatarChange',
    'baseline_HR', 'task_HR', 'avatar_HR',
    'baseline_RMSSD', 'task_RMSSD', 'avatar_RMSSD',
    'delta_HR_task_minus_baseline', 'delta_RMSSD_task_minus_baseline',]]

associations(subset, cmap='coolwarm', nominal_columns=['Anxiety_Group', 'TaskType', 'AvatarType'], numerical_columns=['STAI_Pre', 'STAI_Post', 'STAI_Change',
    'TaskChange', 'AvatarChange',
    'baseline_HR', 'task_HR', 'avatar_HR',
    'baseline_RMSSD', 'task_RMSSD', 'avatar_RMSSD',
    'delta_HR_task_minus_baseline', 'delta_RMSSD_task_minus_baseline'])

#### IPQ Results and Visualisation

In [ ]:
import pandas as pd

df = pd.read_csv("Experiment1 - Form Responses.csv")

question_phrases = [
    "I had the feeling that I was actually there",
    "I felt surrounded by the virtual environment",
    "I had the feeling that I was just watching pictures",  
    "I did not feel present in the virtual space",           
    "I was not just operating something externally",
    "I had the impression of being in the middle of the action",
    "how much were you aware of the real world",            
    "I was no longer aware of my real environment",
    "I was still aware of the real environment",            
    "I was completely captivated by the virtual world",
    "How real did the virtual world seem to you",
    "How much did your experience in the virtual environment resemble",
    "To what extent did the virtual world seem like reality",
    "The virtual world seemed more realistic than the real world"
]

reverse_indices = [2, 3, 6, 8]

question_columns = []
for phrase in question_phrases:
    match = next((col for col in df.columns if phrase in col), None)
    if match:
        question_columns.append(match)
    else:
        raise ValueError(f"Could not find column for phrase: {phrase}")

result = pd.DataFrame()
result["ParticipantID"] = df["学生ID / StudentID"]

for i, col in enumerate(question_columns):
    q_label = f"Q{i+1}"
    if i in reverse_indices:
        result[q_label] = 8 - df[col]
    else:
        result[q_label] = df[col]

result["IPQ_TotalScore"] = result[[f"Q{i+1}" for i in range(14)]].sum(axis=1)

INV_items = ['Q1', 'Q2', 'Q5', 'Q6', 'Q10']
SP_items = ['Q3', 'Q4', 'Q7', 'Q8', 'Q9']
REAL_items = ['Q11', 'Q12', 'Q13', 'Q14']

INV = df[INV_items].mean(axis=1)
SP = df[SP_items].mean(axis=1)
REAL = df[REAL_items].mean(axis=1)

means = [INV.mean(), SP.mean(), REAL.mean()]
errors = [INV.std(ddof=1)/np.sqrt(len(INV)), SP.std(ddof=1)/np.sqrt(len(SP)), REAL.std(ddof=1)/np.sqrt(len(REAL))]

labels = ['Involvement', 'Spatial Presence', 'Realism']
x = np.arange(len(labels))

fig, ax = plt.subplots(figsize=(8,6))
bars = ax.bar(x, means, yerr=errors, capsize=8, color=['#FECF8D', '#F9A87D', '#DC5F5F'], edgecolor='black')

ax.set_ylabel('Mean Rating (1–7)')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_title('Mean Ratings of IPQ Subscales')
ax.set_ylim(0, 8)

for i, data in enumerate([INV, SP, REAL]):
    jitter = np.random.normal(loc=0, scale=0.05, size=len(data))
    ax.scatter(np.full_like(data, x[i]) + jitter, data, color='black', s=20, alpha=0.6)

plt.tight_layout()
plt.show()